## 任务拆解

本节开发一个 **选课通知 Agent**，来实践 **任务拆解** 范式。

我们给这个 Agent 设定的目标是，给没有选课的同学，发送邮件通知。我们预期它能完成以下步骤：

1. 查询学生表，获取没选课的学生的邮箱列表
2. 撰写文案，以学校的口吻通知学生
3. 发送通知邮件（MCP 只打 log，不实际发送）

我们不会用 hard coding 的方式规定 Agent 的执行内容，而是让它自己将目标拆解成关键任务 (core mission)，然后自发执行。

### 1. 构造样本数据

我们要编两张 PostgreSQL 表：学生选课状态表、学生邮箱表。

建表语句：

```sql
-- 创建数据库
CREATE DATABASE course_selection;

-- 创建新用户
CREATE USER admin WITH ENCRYPTED PASSWORD 'admin-password';

-- 授予用户权限
GRANT ALL PRIVILEGES ON DATABASE course_selection TO admin;

-- 切换数据库
\c course_selection

-- 学生选课状态表
CREATE TABLE student (
    log_id SERIAL PRIMARY KEY,            -- 日志ID（主键，自增）
    student_id VARCHAR(20) NOT NULL,      -- 学生ID（非空）
    class VARCHAR(50) NOT NULL,           -- 学生班级（非空）
    course_selected BOOLEAN DEFAULT FALSE,-- 是否完成选课（默认未完成）
    update_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- 更新时间戳（默认当前时间）
);

-- 学生邮箱表
CREATE TABLE email (
    log_id SERIAL PRIMARY KEY,            -- 日志ID（主键，自增）
    student_id VARCHAR(20) NOT NULL,      -- 学生ID（非空）
    email VARCHAR(100) NOT NULL,   -- 学生邮箱（非空）
    update_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- 更新时间戳（默认当前时间）
);

-- 赋予表权限
GRANT SELECT ON student TO admin;
GRANT SELECT ON email TO admin;
```

上面是两张有历史记录的数据表，注意需要取最新的记录使用。接下来我们为它添加注释：

```sql
-- 添加 student 表的注释
COMMENT ON TABLE student IS '学生选课状态表，记录学生的选课完成状态及更新时间';

-- 添加 student 表字段注释
COMMENT ON COLUMN student.log_id IS '日志ID（主键），自增序列，唯一标识每条记录';
COMMENT ON COLUMN student.student_id IS '学生ID，非空，标识唯一学生';
COMMENT ON COLUMN student.class IS '学生所属班级，非空';
COMMENT ON COLUMN student.course_selected IS '选课完成状态：TRUE-已完成选课，FALSE-未完成（默认值）';
COMMENT ON COLUMN student.update_timestamp IS '记录最后更新时间，默认值为当前时间';

-- 添加 email 表的注释
COMMENT ON TABLE email IS '学生邮箱表，记录学生的唯一邮箱信息';

-- 添加 email 表字段注释
COMMENT ON COLUMN email.log_id IS '日志ID（主键），自增序列，唯一标识每条记录';
COMMENT ON COLUMN email.student_id IS '学生ID，非空，与student表关联';
COMMENT ON COLUMN email.email IS '学生邮箱地址，唯一且非空';
COMMENT ON COLUMN email.update_timestamp IS '记录最后更新时间，默认值为当前时间';
```

让 DeepSeek 帮我造一些学生数据：

```sql
-- 插入学生选课状态数据
INSERT INTO student (student_id, class, course_selected, update_timestamp) VALUES
('S001', 'Class A', FALSE, '2024-01-01 09:00:00'),
('S001', 'Class A', TRUE, '2024-01-03 14:30:00'),
('S002', 'Class A', FALSE, '2024-01-01 09:15:00'),
('S002', 'Class A', TRUE, '2024-01-04 11:20:00'),
('S003', 'Class A', FALSE, '2024-01-01 10:00:00'),
('S003', 'Class A', FALSE, '2024-01-02 16:45:00'),
('S003', 'Class A', TRUE, '2024-01-05 10:30:00'),
('S004', 'Class B', FALSE, '2024-01-01 11:30:00'),
('S004', 'Class B', TRUE, '2024-01-03 09:15:00'),
('S005', 'Class B', FALSE, '2024-01-01 13:00:00'),
('S005', 'Class B', FALSE, '2024-01-04 15:20:00'),
('S006', 'Class C', FALSE, '2024-01-01 14:00:00'),
('S006', 'Class C', TRUE, '2024-01-06 12:40:00'),
('S007', 'Class C', FALSE, '2024-01-02 08:45:00'),
('S007', 'Class C', TRUE, '2024-01-05 14:15:00');

-- 插入学生邮箱数据
INSERT INTO email (student_id, email, update_timestamp) VALUES
('S001', 's001_old@university.edu', '2024-01-01 09:05:00'),
('S001', 's001_new@university.edu', '2024-01-04 10:20:00'),
('S002', 's002_initial@university.edu', '2024-01-01 09:20:00'),
('S002', 's002_updated@university.edu', '2024-01-05 11:30:00'),
('S003', 's003_first@university.edu', '2024-01-01 10:10:00'),
('S003', 's003_second@university.edu', '2024-01-03 15:00:00'),
('S003', 's003_final@university.edu', '2024-01-06 09:45:00'),
('S004', 's004_old@university.edu', '2024-01-01 11:35:00'),
('S004', 's004_new@university.edu', '2024-01-04 14:00:00'),
('S005', 's005_primary@university.edu', '2024-01-01 13:05:00'),
('S005', 's005_secondary@university.edu', '2024-01-05 16:10:00'),
('S006', 's006_first@university.edu', '2024-01-01 14:10:00'),
('S006', 's006_official@university.edu', '2024-01-07 10:30:00'),
('S007', 's007_initial@university.edu', '2024-01-02 09:00:00'),
('S007', 's007_updated@university.edu', '2024-01-06 13:20:00');
```
